In [38]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import re
import pandas as pd
import pymongo

In [39]:
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [40]:
print(soup.prettify())

<!DOCTYPE html>
<!--[if lte IE 9]> <p class="browsehappy">You are using an <strong>outdated</strong> browser. Please <a href="http://browsehappy.com/">upgrade your browser</a> to improve your experience.</p> <![endif]-->
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <!-- Always force latest IE rendering engine or request Chrome Frame -->
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <script type="text/javascript">
   window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"5e33925808","applicationID":"59562082","transactionName":"JVcPR0MLWApSRU1eAQVVEhxSC1oSUlkWbBMHXwRAHhdcCUA=","queueTime":0,"applicationTime":165,"agent":""}
  </script>
  <script type="text/javascript">
   (window.NREUM||(NREUM={})).loader_config={xpid:"VQcPUlZTDxAFXVRUBQEPVA=="};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r

In [41]:
title_results = soup.find('div', class_='content_title')
title = title_results.text.strip()

In [42]:
title

'Nearly a Decade After Mars Phoenix Landed, Another Look'

In [43]:
para_results = soup.find('div', class_='rollover_description_inner')
para = para_results.text.strip()

In [44]:
para

"A recent view from Mars orbit of the site where NASA's Phoenix Mars mission landed on far-northern Mars nearly a decade ago captures changes."

In [45]:
mars_p = []
for result in para_results:
    mars_p.append(result.text.strip())

AttributeError: 'NavigableString' object has no attribute 'text'

In [46]:
mars_p

[]

In [47]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [48]:
url = 'https://www.jpl.nasa.gov/spaceimages/index.php?category=Mars'

In [49]:
browser.visit(url)
for x in range(1):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    image = soup.find_all('article', class_='carousel_item')
    #browser.click_link_by_partial_text('FULL IMAGE')

for i in image:
    image_url = i['style']
    print(i["style"])

background-image: url('/spaceimages/images/wallpaper/PIA14925-1920x1200.jpg');


In [50]:
full_image = image_url.split("'")

first_url = url.split("space")

featured_image_url = first_url[0][:-1] + full_image[1]

In [51]:
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA14925-1920x1200.jpg'

In [52]:
twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twitter_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
tweet_text = soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')
mars_weather = tweet_text.text.strip()

In [53]:
mars_weather

'Sol 1970 (Feb 20, 2018), Sunny, high -16C/3F, low -77C/-106F, pressure at 7.32 hPa, daylight 05:38-17:26'

In [54]:
fact_url = 'https://space-facts.com/mars/'
fact_tables = pd.read_html(fact_url)
fact_tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [55]:
fact_df = fact_tables[0]
fact_df.columns = ['Description','Values']
fact_df.set_index('Description', inplace=True)
fact_df

,Values
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [56]:
fact_html = fact_df.to_html()
fact_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Values</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr

In [57]:
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
hemi_titles = ['Cerberus Hemisphere','Schiaparelli Hemisphere','Syrtis Major Hemisphere','Valles Marineris Hemisphere']
hemi_img_links = []
for i in range(4):
    browser.visit(hemi_url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    hemi_text = soup.find('a', class_='itemLink product-item')
    browser.click_link_by_partial_text(hemi_titles[i])
    hemi_url_2 = browser.url
    browser.visit(hemi_url_2)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    hemi_img = soup.find_all('a', href=True)
    img_links = []
    for img in hemi_img:
        img_links.append(img['href'])
    hemi_img_links.append(img_links[41])

In [37]:
img_links[41]

'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'

In [58]:
hemi_img_links

['http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg']

In [59]:
hemisphere_image_urls = [
    {'title':hemi_titles[0],'img_url':hemi_img_links[0]},
    {'title':hemi_titles[1],'img_url':hemi_img_links[1]},
    {'title':hemi_titles[2],'img_url':hemi_img_links[2]},
    {'title':hemi_titles[3],'img_url':hemi_img_links[3]}
]

In [60]:
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere'}]

In [65]:
mars_scrape = {
    "title":title,"paragraph":para,"featured_image":featured_image_url,"weather":mars_weather,"table":fact_html,"hemisphere":hemisphere_image_urls
}

In [25]:
mars_scrape

{'featured image': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA01384-1920x1200.jpg',
 'hemisphere': [{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif',
   'title': 'Cerberus Hemisphere'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif',
   'title': 'Schiaparelli Hemisphere'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif',
   'title': 'Syrtis Major Hemisphere'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif',
   'title': 'Valles Marineris Hemisphere'}],
 'paragraph': "A recent view from Mars orbit of the site where NASA's Phoenix Mars mission landed on far-northern Mars nearly a decade ago captures changes.",
 'table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Values</th>\n    </tr>\n   

In [66]:
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db = client.mars

In [67]:
db.mars.insert_one(mars_scrape)

In [32]:
db.mars.find()

In [64]:
for document in db.mars.find():
    print(document)

{'_id': ObjectId('5a94bdee2a9e9535d448884c'), 'title': 'Nearly a Decade After Mars Phoenix Landed, Another Look', 'paragraph': "A recent view from Mars orbit of the site where NASA's Phoenix Mars mission landed on far-northern Mars nearly a decade ago captures changes.", 'featured image': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA14925-1920x1200.jpg', 'weather': 'Sol 1970 (Feb 20, 2018), Sunny, high -16C/3F, low -77C/-106F, pressure at 7.32 hPa, daylight 05:38-17:26', 'table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Values</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons

In [35]:
type(fact_html)

str